<a href="https://colab.research.google.com/github/SofiiaHeryha/Portfolio/blob/Python-projects/AB_test_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade google-cloud-bigquery
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
client = bigquery.Client(project="data-analytics-mate")

query = """
WITH session_info as (
  SELECT s.date,
         s.ga_session_id,
         sp.country,
         sp.device,
         sp.continent,
         sp.channel,
         ab.test,
         ab.test_group
  FROM `DA.ab_test` ab
  JOIN `DA.session` s ON ab.ga_session_id = s.ga_session_id
  JOIN `DA.session_params` sp ON sp.ga_session_id = ab.ga_session_id),
     session_with_orders AS (
  SELECT session_info.date,
         session_info.country,
         session_info.device,
         session_info.continent,
         session_info.channel,
         session_info.test,
         session_info.test_group,
         COUNT(DISTINCT o.ga_session_id) AS session_with_orders
  FROM `DA.order` o
  JOIN session_info ON  o.ga_session_id = session_info.ga_session_id
  GROUP BY
         session_info.date,
         session_info.country,
         session_info.device,
         session_info.continent,
         session_info.channel,
         session_info.test,
         session_info.test_group),
     events AS (
  SELECT session_info.date,
         session_info.country,
         session_info.device,
         session_info.continent,
         session_info.channel,
         session_info.test,
         session_info.test_group,
         ep.event_name,
         COUNT(ep.ga_session_id) AS event_cnt
  FROM `DA.event_params` ep
  JOIN session_info ON ep.ga_session_id = session_info.ga_session_id
  GROUP BY
         session_info.date,
         session_info.country,
         session_info.device,
         session_info.continent,
         session_info.channel,
         session_info.test,
         session_info.test_group,
         ep.event_name),
     session AS(
  SELECT session_info.date,
         session_info.country,
         session_info.device,
         session_info.continent,
         session_info.channel,
         session_info.test,
         session_info.test_group,
         COUNT( DISTINCT session_info.ga_session_id) AS session_cnt
  FROM session_info
  GROUP BY
         session_info.date,
         session_info.country,
         session_info.device,
         session_info.continent,
         session_info.channel,
         session_info.test,
         session_info.test_group),
     account AS (
  SELECT session_info.date,
         session_info.country,
         session_info.device,
         session_info.continent,
         session_info.channel,
         session_info.test,
         session_info.test_group,
         COUNT(DISTINCT acs.ga_session_id) AS new_account_cnt
  FROM `DA.account_session` acs
  JOIN session_info ON acs.ga_session_id = session_info.ga_session_id
  GROUP BY
         session_info.date,
         session_info.country,
         session_info.device,
         session_info.continent,
         session_info.channel,
         session_info.test,
         session_info.test_group)
  SELECT session_with_orders.date,
         session_with_orders.country,
         session_with_orders.device,
         session_with_orders.continent,
         session_with_orders.channel,
         session_with_orders.test,
         session_with_orders.test_group,
        'session with orders' AS event_name,
         session_with_orders.session_with_orders AS value,
  FROM session_with_orders
UNION ALL
  SELECT events.date,
         events.country,
         events.device,
         events.continent,
         events.channel,
         events.test,
         events.test_group,
         events.event_name,
         events.event_cnt AS value,
  FROM events
UNION ALL
  SELECT session.date,
         session.country,
         session.device,
         session.continent,
         session.channel,
         session.test,
         session.test_group,
         'session' AS event_name,
         session_cnt AS value,
  FROM session
UNION ALL
  SELECT account.date,
         account.country,
         account.device,
         account.continent,
         account.channel,
         account.test,
         account.test_group,
         'new account' AS event_name,
         new_account_cnt AS value,
  FROM account
"""
query_job = client.query(query)
results = query_job.result()
df = results.to_dataframe()
df.to_csv('data_fr.csv', index=False)
df.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-03,Belarus,desktop,Europe,Paid Search,2,1,session with orders,1
1,2020-11-04,Guatemala,desktop,Americas,Undefined,2,1,session with orders,1
2,2020-11-05,Jamaica,mobile,Americas,Organic Search,2,1,session with orders,1
3,2020-11-06,Cambodia,desktop,Asia,Organic Search,2,1,session with orders,1
4,2020-11-07,Qatar,desktop,Asia,Paid Search,2,2,session with orders,1


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

In [ ]:
metrics = {
    "add_payment_info": "session",
    "add_shipping_info": "session",
    "begin_checkout": "session",
    "new account": "session"}
sessions = df[df["event_name"] == "session"] \
    .groupby(["test", "test_group"])["value"].sum().reset_index()

In [ ]:
results = []

for metric, denom in metrics.items():

    metric_df = df[df["event_name"] == metric] \
        .groupby(["test", "test_group"])["value"].sum().reset_index()

    merged = pd.merge(metric_df, sessions,
                      on=["test", "test_group"],
                      suffixes=("_metric", "_session"))

    for test_id in merged["test"].unique():

        tmp = merged[merged["test"] == test_id]

        if len(tmp) != 2:
            continue

        a = tmp.iloc[0]
        b = tmp.iloc[1]

        n1, d1 = a["value_metric"], a["value_session"]
        n2, d2 = b["value_metric"], b["value_session"]

        cr1 = n1 / d1
        cr2 = n2 / d2

        p_pool = (n1 + n2) / (d1 + d2)
        se = np.sqrt(p_pool * (1 - p_pool) * (1/d1 + 1/d2))
        z = (cr1 - cr2) / se
        p_value = 2 * (1 - stats.norm.cdf(abs(z)))

        results.append([
            test_id,
            metric,
            metric,
            "session",
            n1, d1, cr1,
            n2, d2, cr2,
            (cr2-cr1)/cr1,
            z,
            p_value,
            p_value < 0.05])

In [ ]:
cols = [
    "test_number","metric","numerator","denominator",
    "numerator_control","denominator_control","conversion_rate_control",
    "numerator_test","denominator_test","conversion_rate_test",
    "metric_change","z_stat","p_value","significant"]

final_df = pd.DataFrame(results, columns=cols)
final_df["conversion_rate_control"] = final_df["conversion_rate_control"].round(6)
final_df["conversion_rate_test"] = final_df["conversion_rate_test"].mul(100).round(2)
final_df["metric_change"] = final_df["metric_change"].mul(100).round(2)
final_df["z_stat"] = final_df["z_stat"].round(3)
final_df["p_value"] = final_df["p_value"].round(3)

final_df

,test_number,metric,numerator,denominator,numerator_control,denominator_control,conversion_rate_control,numerator_test,denominator_test,conversion_rate_test,metric_change,z_stat,p_value,significant
0,1,add_payment_info,add_payment_info,session,1988,45362,0.043825,2229,45193,4.93,12.54,-3.925,0.000,True
1,2,add_payment_info,add_payment_info,session,2344,50637,0.046290,2409,50244,4.79,3.58,-1.241,0.215,False
2,3,add_payment_info,add_payment_info,session,3623,70047,0.051722,3697,70439,5.25,1.47,-0.643,0.520,False
3,4,add_payment_info,add_payment_info,session,3731,105079,0.035507,3601,105141,3.42,-3.54,1.571,0.116,False
4,1,add_shipping_info,add_shipping_info,session,3034,45362,0.066884,3221,45193,7.13,6.56,-2.604,0.009,True
5,2,add_shipping_info,add_shipping_info,session,3480,50637,0.068724,3510,50244,6.99,1.65,-0.710,0.478,False
6,3,add_shipping_info,add_shipping_info,session,5298,70047,0.075635,5188,70439,7.37,-2.62,1.414,0.157,False
7,4,add_shipping_info,add_shipping_info,session,5128,105079,0.048801,4956,105141,4.71,-3.41,1.786,0.074,False
8,1,begin_checkout,begin_checkout,session,3784,45362,0.083418,4021,45193,8.90,6.66,-2.979,0.003,True
9,2,begin_checkout,begin_checkout,session,4262,50637,0.084168,4313,50244,8.58,1.99,-0.953,0.341,False


In [ ]:
final_df.to_csv("/content/drive/MyDrive/data.csv",
                index=False,
                sep=';')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## [Results](https://drive.google.com/file/d/1MvCnybR7WMfuX2fHejLT-dQYo4mBto9J/view?usp=drive_link)



## [Dashboard](https://public.tableau.com/views/ABTest_17550147242240/ABtest?:language=en-US&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link)
